In [1]:
import numpy as np
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from natsort import natsorted
import math
import pandas as pd
from nltk.stem import PorterStemmer


positional_index = {}

# Preprocessing and Read Data

In [2]:
def read_and_tokenize_documents(directory_path):
    files_name = natsorted(os.listdir(directory_path))
    token_lists = []
    term_lists = []

    for file_name in files_name:
        with open(os.path.join(directory_path, file_name), "r") as f:
            document = f.read()
            tokens, terms = tokenize_and_stem(document)
            token_lists.append(tokens)
            term_lists.append(terms)

    return token_lists, term_lists

def tokenize_and_stem(doc):
    token_docs = word_tokenize(doc)
    prepared_tokens = [token.lower() for token in token_docs]
    stemmed_terms = [stem(token) for token in token_docs]
    return prepared_tokens, stemmed_terms

def stem(token):
    return PorterStemmer().stem(token.lower())

In [3]:
docs_directory_path = "docs"
document_of_tokens, document_of_terms = read_and_tokenize_documents(docs_directory_path)
document_of_terms

[['antoni', 'brutu', 'caeser', 'cleopatra', 'merci', 'worser'],
 ['antoni', 'brutu', 'caeser', 'calpurnia'],
 ['merci', 'worser'],
 ['brutu', 'caeser', 'merci', 'worser'],
 ['caeser', 'merci', 'worser'],
 ['antoni', 'caeser', 'merci'],
 ['angel', 'fool', 'fear', 'in', 'rush', 'to', 'tread', 'where'],
 ['angel', 'fool', 'fear', 'in', 'rush', 'to', 'tread', 'where'],
 ['angel', 'fool', 'in', 'rush', 'to', 'tread', 'where'],
 ['fool', 'fear', 'in', 'rush', 'to', 'tread', 'where']]

# POSITIONAL INDEX MODEL

In [4]:
def build_positional_index(document_of_terms):
    for doc_id, document in enumerate(document_of_terms, start=1):
        for position, term in enumerate(document):
            if term not in positional_index:
                positional_index[term] = [0, {}]
            positional_index[term][0] += 1
            positional_index[term][1].setdefault(doc_id, []).append(position)
    return positional_index

def get_key_by_value(dictionary, target_value, default=None):
    """Get the key for a given value in a dictionary."""
    return next((key for key, value in dictionary.items() if value == target_value), default)

In [5]:
build_positional_index(document_of_terms)

{'antoni': [3, {1: [0], 2: [0], 6: [0]}],
 'brutu': [3, {1: [1], 2: [1], 4: [0]}],
 'caeser': [5, {1: [2], 2: [2], 4: [1], 5: [0], 6: [1]}],
 'cleopatra': [1, {1: [3]}],
 'merci': [5, {1: [4], 3: [0], 4: [2], 5: [1], 6: [2]}],
 'worser': [4, {1: [5], 3: [1], 4: [3], 5: [2]}],
 'calpurnia': [1, {2: [3]}],
 'angel': [3, {7: [0], 8: [0], 9: [0]}],
 'fool': [4, {7: [1], 8: [1], 9: [1], 10: [0]}],
 'fear': [3, {7: [2], 8: [2], 10: [1]}],
 'in': [4, {7: [3], 8: [3], 9: [2], 10: [2]}],
 'rush': [4, {7: [4], 8: [4], 9: [3], 10: [3]}],
 'to': [4, {7: [5], 8: [5], 9: [4], 10: [4]}],
 'tread': [4, {7: [6], 8: [6], 9: [5], 10: [5]}],
 'where': [4, {7: [7], 8: [7], 9: [6], 10: [6]}]}

# TERM-FREQUANCY.TF

In [6]:
def create_term_frequency_dataframe(document_of_tokens):
    term_frequency = {}

    for doc_id, document in enumerate(document_of_tokens, start=1):
        for term in document:
            term_frequency.setdefault(term, {}).setdefault(doc_id, 0)
            term_frequency[term][doc_id] += 1

    df = pd.DataFrame.from_dict(term_frequency, orient='index').fillna(0)
    df.columns = [f'doc_{col}' for col in df.columns]

    return df

In [7]:
document_of_terms

[['antoni', 'brutu', 'caeser', 'cleopatra', 'merci', 'worser'],
 ['antoni', 'brutu', 'caeser', 'calpurnia'],
 ['merci', 'worser'],
 ['brutu', 'caeser', 'merci', 'worser'],
 ['caeser', 'merci', 'worser'],
 ['antoni', 'caeser', 'merci'],
 ['angel', 'fool', 'fear', 'in', 'rush', 'to', 'tread', 'where'],
 ['angel', 'fool', 'fear', 'in', 'rush', 'to', 'tread', 'where'],
 ['angel', 'fool', 'in', 'rush', 'to', 'tread', 'where'],
 ['fool', 'fear', 'in', 'rush', 'to', 'tread', 'where']]

In [8]:
term_freq_df = create_term_frequency_dataframe(document_of_tokens)
term_freq_df

,doc_1,doc_2,doc_6,doc_4,doc_5,doc_3,doc_7,doc_8,doc_9,doc_10
antony,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
brutus,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
caeser,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
cleopatra,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mercy,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
worser,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
calpurnia,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
angels,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
fools,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
fear,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0


# WEIGHTED TF TABLE

In [9]:
def weighted_TF(x):
    return np.log10(x) + 1 if x > 0 else 0

weighted_term_freq_df = term_freq_df.apply(lambda x: x.apply(weighted_TF))
weighted_term_freq_df

,doc_1,doc_2,doc_6,doc_4,doc_5,doc_3,doc_7,doc_8,doc_9,doc_10
antony,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
brutus,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
caeser,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
cleopatra,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mercy,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
worser,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
calpurnia,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
angels,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
fools,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
fear,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0


# INVERSE DOCUMENT FREQUANCY IDF 

In [10]:
def calculate_doc_frequency_doc_inv_frequency(term_freq_df):
    number_of_docs = len(term_freq_df.columns)
    df = term_freq_df.sum(axis=1)
    inverse_doc_freq = np.log10(number_of_docs / df.astype(float))

    idf = pd.DataFrame(
        {"doc_freq": df, "inverse_doc_freq": inverse_doc_freq},
        index=term_freq_df.index,
    )
    return idf

In [11]:
tfdf = calculate_doc_frequency_doc_inv_frequency(term_freq_df)
tfdf

,doc_freq,inverse_doc_freq
antony,3.0,0.522879
brutus,3.0,0.522879
caeser,5.0,0.301030
cleopatra,1.0,1.000000
mercy,5.0,0.301030
worser,4.0,0.397940
calpurnia,1.0,1.000000
angels,3.0,0.522879
fools,4.0,0.397940
fear,3.0,0.522879


# TF.IDF

In [12]:
def calculate_tf_idf_optimized(term_freq_df, idf):
    term_freq_inverse_doc_freq = term_freq_df.mul(idf["inverse_doc_freq"], axis=0)
    return term_freq_inverse_doc_freq

In [13]:
tfidf = calculate_tf_idf_optimized(term_freq_df, tfdf)
tfidf

,doc_1,doc_2,doc_6,doc_4,doc_5,doc_3,doc_7,doc_8,doc_9,doc_10
antony,0.522879,0.522879,0.522879,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
brutus,0.522879,0.522879,0.000000,0.522879,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
caeser,0.301030,0.301030,0.301030,0.301030,0.30103,0.00000,0.000000,0.000000,0.000000,0.000000
cleopatra,1.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
mercy,0.301030,0.000000,0.301030,0.301030,0.30103,0.30103,0.000000,0.000000,0.000000,0.000000
worser,0.397940,0.000000,0.000000,0.397940,0.39794,0.39794,0.000000,0.000000,0.000000,0.000000
calpurnia,0.000000,1.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000
angels,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.522879,0.522879,0.522879,0.000000
fools,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.397940,0.397940,0.397940,0.397940
fear,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.522879,0.522879,0.000000,0.522879


# COSINE SIMILARITY, DOCUMENT LENGTHS, and NORMALIZATION 

In [14]:
def cosine_similarity(query_vector, document_vectors):
    query_magnitude = np.linalg.norm(query_vector)
    document_magnitudes = np.linalg.norm(document_vectors, axis=1)

    dot_products = np.dot(document_vectors, query_vector)
    cosine_similarities = dot_products / (document_magnitudes * query_magnitude)

    return cosine_similarities


def get_query_vector(query, all_words):
    query_vector = np.zeros(len(all_words))
    query_terms = query.split()

    for term in query_terms:
        if term in all_words:
            query_vector[all_words.index(term)] += 1

    return query_vector


def calculate_document_lengths(term_freq_inve_doc_freq):
    return np.sqrt((term_freq_inve_doc_freq**2).sum(axis=0))


def normalize_term_freq_idf(term_freq_inve_doc_freq, document_lengths):
    return term_freq_inve_doc_freq.div(document_lengths, axis=1, level=1)

In [15]:
document_lengths = calculate_document_lengths(tfidf)
document_lengths

doc_1     1.373462
doc_2     1.279618
doc_6     0.674270
doc_4     0.782941
doc_5     0.582747
doc_3     0.498974
doc_7     1.223496
doc_8     1.223496
doc_9     1.106137
doc_10    1.106137
dtype: float64

In [16]:
normalized_term_freq_idf = normalize_term_freq_idf(tfidf, document_lengths)
normalized_term_freq_idf

,doc_1,doc_2,doc_6,doc_4,doc_5,doc_3,doc_7,doc_8,doc_9,doc_10
antony,0.380701,0.408621,0.775474,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
brutus,0.380701,0.408621,0.000000,0.667839,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
caeser,0.219176,0.235250,0.446453,0.384486,0.516570,0.000000,0.000000,0.000000,0.000000,0.000000
cleopatra,0.728087,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
mercy,0.219176,0.000000,0.446453,0.384486,0.516570,0.603298,0.000000,0.000000,0.000000,0.000000
worser,0.289735,0.000000,0.000000,0.508263,0.682869,0.797516,0.000000,0.000000,0.000000,0.000000
calpurnia,0.000000,0.781483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
angels,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.427365,0.427365,0.472707,0.000000
fools,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.325248,0.325248,0.359756,0.359756
fear,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.427365,0.427365,0.000000,0.472707


# TEST THE SEARCH ENGINE and SAVE RESULT IN TXT FILE 

In [17]:
def create_query_dataframe(query_terms, normalized_term_freq_idf, tfdf, positional_index):
    query_df = pd.DataFrame(index=normalized_term_freq_idf.index)
    query_df["tf"] = [1 if term in query_terms else 0 for term in positional_index.keys()]
    query_df["w_tf"] = query_df["tf"].apply(weighted_TF)
    query_df["idf"] = tfdf["inverse_doc_freq"] * query_df["w_tf"]
    query_df["tf_idf"] = query_df["w_tf"] * query_df["idf"]
    query_df["normalized"] = query_df["idf"] / np.sqrt((query_df["idf"] ** 2).sum())
    return query_df


def calculate_product(query_df, normalized_term_freq_idf):
    product = normalized_term_freq_idf.multiply(query_df["normalized"], axis=0)
    return product


def calculate_cosine_similarity(product_result):
    return product_result.sum()


def get_related_docs(query_df, positional_index):
    related_docs = set()

    for term in query_df[query_df["tf"] > 0].index:
        if term in positional_index:
            related_docs.update(positional_index[term][1].keys())

    return list(related_docs)


def write_to_file(file_path, content):
    with open(file_path, "w") as file:
        file.write(content)
    print("Your results are printed :)")

#                            PHRASE QUERY

In [18]:
def find_matching_positions(queries, positional_index):
    
    final_result = []
    for query in queries:
        term_lists = [[] for _ in range(10)]
        _, query_terms = tokenize_and_stem(query)
        if query_terms:
            for term in query_terms:
                if term not in positional_index:
                    return False
                else:
                    for key, positions in positional_index[term][1].items():
                        term_lists[key - 1].extend(positions)

            matching_positions = [
                f"{pos}"
                for pos, positions in enumerate(term_lists, start=1)
                if len(positions) == len(query_terms)
            ]

            final_result.append(matching_positions)
        else:
            return False
    return final_result

In [19]:
query = ["antony brutus", "mercy worser"]
matching = find_matching_positions(["antony brutus", "mercy worser"], positional_index)

In [20]:
if "and" in query:
    set(matching[0]).intersection(set(matching[1]))
        

In [21]:
set(matching[0]).intersection(set(matching[1]))

{'1'}

In [22]:
matching

[['1', '2'], ['1', '3', '4', '5']]

In [23]:
def find_matching_positions(queries, positional_index):
    final_result = []
    for query in queries:
        term_lists = [[] for _ in range(10)]
        _, query_terms = tokenize_and_stem(query)
        if query_terms:
            for term in query_terms:
                if term not in positional_index:
                    return False
                else:
                    for key, positions in positional_index[term][1].items():
                        term_lists[key - 1].extend(positions)

            matching_positions = [
                f"{pos}"
                for pos, positions in enumerate(term_lists, start=1)
                if len(positions) == len(query_terms)
            ]

            final_result.append(matching_positions)
        else:
            return False
    return final_result


In [24]:
def and_boolean_query(returned_matches_docs):
    if not returned_matches_docs or any(not doc_set for doc_set in returned_matches_docs):
        print("No matched documents.")
        return None

    temp_result = set(returned_matches_docs[0])

    for doc_set in returned_matches_docs[1:]:
        temp_result = temp_result.intersection(set(doc_set))

    return list(temp_result)

In [25]:
def or_boolean_query(returned_matches_docs):
    if not returned_matches_docs or any(not doc_set for doc_set in returned_matches_docs):
        print("No matched documents.")
        return None

    temp_result = set()

    for doc_set in returned_matches_docs:
        temp_result.update(doc_set)

    return list(temp_result)

In [50]:
def complement_boolean_query(query_set):
    full_set = set(["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
    if not query_set:
        print("No query set provided.")
        return list(full_set)

    complement_set = full_set.difference(query_set)
    return list(complement_set)


In [27]:
def check_contains_boolean_logic(query):
    query_words = query.lower().split(" ")
    boolean_operator = None

    if "and" in query_words:
        boolean_operator = "and"
    elif "or" in query_words:
        boolean_operator = "or"
    elif "not" in query_words:
        boolean_operator = "not"
    query_words = " ".join(query_words)

    return query_words, boolean_operator


In [28]:
def phrase_query_serach(query, positional_index):
    term_lists = [[] for _ in range(10)]
    _, query_terms = tokenize_and_stem(query)
    if query_terms:
        for term in query_terms:
            if term not in positional_index:
                return False
            else:
                for key, positions in positional_index[term][1].items():
                    term_lists[key - 1].extend(positions)

        matching_positions = [
            f"doc_{pos}"
            for pos, positions in enumerate(term_lists, start=1)
            if len(positions) == len(query_terms)
        ]

        return ", ".join(matching_positions)
    else:
        return False

In [31]:
def build_output_content(query, related_docs_PQ_stage):
    query_tokens, query_terms = tokenize_and_stem(query)

    if related_docs_PQ_stage:
        # save the result for Phrase query
        document_lengths = calculate_document_lengths(tfidf)
        normalized_term_freq_idf = normalize_term_freq_idf(tfidf, document_lengths)
        query_df = create_query_dataframe(
            query_terms, normalized_term_freq_idf, tfdf, positional_index
        )
        product_result = calculate_product(query_df, normalized_term_freq_idf)
        similarity = calculate_cosine_similarity(product_result)

        try:
            query_detailed = query_df.loc[query_tokens]

            # Write results to a text file
            results_content = f"""
Vector Space Model for Query:\n{query_detailed}\n\n
Product Sum:\n{(product_result.sum()).loc[related_docs_PQ_stage.split(", "),]}\n\n
Product (query * matched doc):\n{product_result.loc[query_tokens, related_docs_PQ_stage.split(", ")]}\n\n
Similarity:\n{similarity.loc[related_docs_PQ_stage.split(", "),]}\n\n
Query Length:\n{math.sqrt(sum(query_df['idf'] ** 2))}\n\n"""
            results_content += f"\n\nRelated Docs:\n{related_docs_PQ_stage}"
            print(results_content)

        except KeyError:
            print(f"No such query found in the database:{query_tokens}\nTry Again.\n")
    else:
        results_content = f"No such query found in the database:{query_terms}"
        print(results_content)

In [57]:
end_search = ""
while end_search not in ["q", "Q"]:
    query = input("In the Phrase Query stage Search for: ")
    query, boolean_operator = check_contains_boolean_logic(query)
    if boolean_operator == "and":
        query = query.split(" and ")
        returned_matches_doc = find_matching_positions(query, positional_index)
        if returned_matches_doc:
            returned_docs = and_boolean_query(returned_matches_doc)
            returned_docs = "doc_" + ", doc_".join(returned_docs)
            query = " ".join(query)
            build_output_content(query, returned_docs)

    elif boolean_operator == "or":
        query = query.split(" or ")
        returned_matches_doc = find_matching_positions(query, positional_index)
        if returned_matches_doc:
            returned_docs = or_boolean_query(returned_matches_doc)
            returned_docs = "doc_" + ", doc_".join(returned_docs)
            query = " ".join(query)
            build_output_content(query, returned_docs)
    elif boolean_operator == "not ":
        query = query.split(" not ")
        returned_matches_doc = find_matching_positions(query, positional_index)
        returned_docs = complement_boolean_query(returned_matches_doc)
        print(returned_docs)
    else:
        returned_matches_doc = phrase_query_serach(query, positional_index)
        if returned_matches_doc:
            build_output_content(query, returned_matches_doc)
        
    end_search = input("If you want to EXIT enter q/Q: ")


In the Phrase Query stage Search for: antony brutus

Vector Space Model for Query:
        tf  w_tf       idf    tf_idf  normalized
antony   1   1.0  0.522879  0.522879    0.707107
brutus   1   1.0  0.522879  0.522879    0.707107


Product Sum:
doc_1    0.538393
doc_2    0.577877
dtype: float64


Product (query * matched doc):
           doc_1     doc_2
antony  0.269196  0.288939
brutus  0.269196  0.288939


Similarity:
doc_1    0.538393
doc_2    0.577877
dtype: float64


Query Length:
0.7394622130520805



Related Docs:
doc_1, doc_2
If you want to EXIT enter q/Q: antony or to
In the Phrase Query stage Search for: antony or to

Vector Space Model for Query:
        tf  w_tf       idf    tf_idf  normalized
antony   1   1.0  0.522879  0.522879    0.795757
to       1   1.0  0.397940  0.397940    0.605616


Product Sum:
doc_8     0.196976
doc_2     0.325163
doc_10    0.217874
doc_6     0.617089
doc_1     0.302946
doc_7     0.196976
doc_9     0.217874
dtype: float64


Product (query * match

In [43]:
"doc_" + ", doc_".join(['1', '2', '3'])

'doc_1, doc_2, doc_3'